# Assignment 1

In [32]:
# Usual Imports

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import re

In [3]:
food_inspection_data = pd.read_csv("../data/Food_Inspections.csv")
food_inspection_data.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2563767,CERES' TABLE,CERES' TABLE,2137239.0,Restaurant,Risk 1 (High),3124 N BROADWAY,CHICAGO,IL,60657.0,09/15/2022,Non-Inspection,No Entry,NaN,41.938412,-87.644492,"(41.93841197166044, -87.64449180403412)"
1,2563480,FINKI ELEMENTARY ACADEMY,FINKI ELEMENTARY ACADEMY,23541.0,School,Risk 1 (High),2332 S Western (2400W),CHICAGO,IL,60608.0,09/12/2022,Canvass,Pass,NaN,NaN,NaN,NaN
2,2563279,SH GRILL,SH GRILL,2863054.0,Catering,Risk 1 (High),167 N GREEN ST,CHICAGO,IL,60607.0,09/06/2022,License,Pass,NaN,41.885044,-87.648476,"(41.88504417665877, -87.64847590715003)"
3,2562053,SUBWAY,SUBWAY,2863267.0,Restaurant,Risk 1 (High),4351 W ARMITAGE AVE,CHICAGO,IL,60639.0,08/12/2022,License,Pass,NaN,41.916939,-87.735947,"(41.9169392924408, -87.73594733406117)"
4,2560592,PROVARE CHICAGO,PROVARE CHICAGO,2796734.0,Restaurant,Risk 1 (High),1421 W CHICAGO AVE,CHICAGO,IL,60642.0,07/12/2022,Canvass,Pass,NaN,41.896025,-87.663272,"(41.89602502708334, -87.66327229348566)"


In [6]:
food_inspection_data["Results"].unique()

array(['No Entry', 'Pass', 'Out of Business', 'Not Ready',
       'Pass w/ Conditions', 'Fail', 'Business Not Located'], dtype=object)

In [17]:
failed_inspection_data = food_inspection_data[(food_inspection_data["Results"]=="Fail") & (food_inspection_data["Violations"].notna())]

In [71]:
failed_inspection_data.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
36,2544891,"NEW FURAMA RESTAURANT, INC.",NEW FURAMA SEAFOOD RESTAURANT,29331.0,Restaurant,Risk 1 (High),2828 S WENTWORTH AVE,CHICAGO,IL,60616.0,12/01/2021,Complaint,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.841723,-87.631918,"(41.84172282437074, -87.63191819949601)"
52,2534457,YOLK CAFE,YOLK CAFE,2298934.0,Restaurant,Risk 1 (High),500 W MADISON ST,CHICAGO,IL,60661.0,10/07/2021,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.881994,-87.639759,"(41.88199433820508, -87.6397586848809)"
64,2531243,Farm Super Market,Farm Super Market,2017764.0,Grocery Store,Risk 2 (Medium),2255-2257 W Devon AVE,CHICAGO,IL,60659.0,09/10/2021,Complaint Re-Inspection,Fail,"6. PROPER EATING, TASTING, DRINKING, OR TOBACC...",41.997668,-87.687147,"(41.99766818403645, -87.68714725060154)"
74,2528631,SLIMS,SLIMS,2732608.0,Restaurant,Risk 1 (High),2020 W MONTROSE AVE,CHICAGO,IL,60618.0,08/26/2021,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.961556,-87.679862,"(41.96155579127668, -87.67986227331792)"
75,2528618,GO FISH GROUP INC,GO FISH,2757989.0,Restaurant,Risk 2 (Medium),3022 W MADISON,CHICAGO,IL,60612.0,08/26/2021,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.881126,-87.702237,"(41.881126474536984, -87.70223661787263)"


In [87]:
def extract_descriptions(violation):
    split_violations = violation.split(sep="|")
    regex_values = []
    pattern = "(?<=\d.).*(?= - )"
    for i in split_violations:
        regex = re.search(pattern,i)
        if regex is None:
            regex_values.append(re.sub("(\d+.)","",i).strip())
        else:
            regex_values.append(i[regex.start(0):regex.end(0)].strip())
    return '|'.join(regex_values)

In [94]:
description_extract_df = failed_inspection_data["Violations"].apply(extract_descriptions)
description_extract_df  = description_extract_df.str.get_dummies(sep = "|")
description_extract_df.info()

In [84]:
description_extract_df = pd.DataFrame(description_extract_df.tolist())
description_extract_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,"MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPL...",PROCEDURES FOR RESPONDING TO VOMITING AND DIAR...,. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. PROPER HOT HOLDING TEMPERATURES,. PROPER COOLING METHODS USED; ADEQUATE EQUIPM...,. THERMOMETERS PROVIDED & ACCURATE,. FOOD PROPERLY LABELED; ORIGINAL CONTAINER,". INSECTS, RODENTS, & ANIMALS NOT PRESENT",". INSECTS, RODENTS, & ANIMALS NOT PRESENT",. SINGLE-USE/SINGLE-SERVICE ARTICLES: PROPERLY...,...,None,None,None,None,None,None,None,None,None,None
1,"MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPL...",PROCEDURES FOR RESPONDING TO VOMITING AND DIAR...,. PROPER COLD HOLDING TEMPERATURES,. PROPER COOLING METHODS USED; ADEQUATE EQUIPM...,. THERMOMETERS PROVIDED & ACCURATE,". INSECTS, RODENTS, & ANIMALS NOT PRESENT",. WIPING CLOTHS: PROPERLY USED & STORED,". UTENSILS, EQUIPMENT & LINENS: PROPERLY STORE...",". FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, ...",". PHYSICAL FACILITIES INSTALLED, MAINTAINED & ...",...,None,None,None,None,None,None,None,None,None,None
2,"PROPER EATING, TASTING, DRINKING, OR TOBACCO USE",. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. COMPLIANCE WITH VARIANCE/SPECIALIZED PROCESS...,". INSECTS, RODENTS, & ANIMALS NOT PRESENT",". FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, ...",". FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, ...",. NON-FOOD/FOOD CONTACT SURFACES CLEAN,". TOILET FACILITIES: PROPERLY CONSTRUCTED, SUP...",. GARBAGE & REFUSE PROPERLY DISPOSED; FACILITI...,...,None,None,None,None,None,None,None,None,None,None
3,"MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPL...",PROCEDURES FOR RESPONDING TO VOMITING AND DIAR...,. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. PROPER COLD HOLDING TEMPERATURES,. PROPER COOLING METHODS USED; ADEQUATE EQUIPM...,. THERMOMETERS PROVIDED & ACCURATE,". FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, ...",". FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, ...",". WAREWASHING FACILITIES: INSTALLED, MAINTAINE...",. PLUMBING INSTALLED; PROPER BACKFLOW DEVICES,...,None,None,None,None,None,None,None,None,None,None
4,"PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLED...",CITY OF CHICAGO FOOD SERVICE SANITATION CERTIF...,"MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPL...",. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED...,. FOOD PROPERLY LABELED; ORIGINAL CONTAINER,". INSECTS, RODENTS, & ANIMALS NOT PRESENT",". FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, ...",...,None,None,None,None,None,None,None,None,None,None
